### This notebook uses graph laplacian constraint to retrieve the block diagonal matrix, given a noisy version of said matrix

Works 100% on seeds 0-4. 

Gradient-based / differentiable version of the k-BDMS projection in Section 4.2 of http://openaccess.thecvf.com/content_cvpr_2014/papers/Feng_Robust_Subspace_Segmentation_2014_CVPR_paper.pdf.

Requires k (connected components) to be known, but maybe this can be relaxed somehow. 

In [1]:
import numpy as np
import torch
import scipy.linalg as slinalg
from scipy.sparse import csgraph
np.random.seed(0)

### First lets make a noisy block diagonal matrix (note: entries should be positive)

In [2]:
splits = [4, 3, 2]
dims = sum(splits)
noise = 5e-2

In [3]:
components = []
for split in splits:
  components.append(np.ones((split, split)))
block_diag_mask = slinalg.block_diag(*components)
base_matrix = np.random.uniform(size=(dims, dims)) * block_diag_mask
noise_matrix = np.random.randn(dims, dims) * noise
noisy_matrix = np.abs(base_matrix + noise_matrix).round(2)
noisy_matrix

array([[0.52, 0.71, 0.63, 0.63, 0.01, 0.09, 0.01, 0.04, 0.01],
       [0.41, 0.75, 0.55, 0.52, 0.05, 0.02, 0.06, 0.05, 0.05],
       [0.87, 0.95, 0.91, 0.82, 0.  , 0.03, 0.02, 0.04, 0.05],
       [0.92, 0.57, 0.4 , 0.31, 0.01, 0.05, 0.05, 0.03, 0.  ],
       [0.05, 0.09, 0.01, 0.  , 0.41, 0.37, 0.74, 0.07, 0.05],
       [0.01, 0.03, 0.09, 0.08, 0.46, 0.63, 0.5 , 0.05, 0.01],
       [0.05, 0.1 , 0.  , 0.01, 0.44, 0.26, 0.17, 0.02, 0.  ],
       [0.  , 0.05, 0.01, 0.02, 0.02, 0.1 , 0.07, 0.99, 0.45],
       [0.09, 0.02, 0.07, 0.04, 0.  , 0.06, 0.03, 0.06, 0.33]])

### Observe that randomly thresholding isn't super effective:

In [4]:
thresholded = noisy_matrix.copy()
thresholded[thresholded < 0.05] = 0.
thresholded

array([[0.52, 0.71, 0.63, 0.63, 0.  , 0.09, 0.  , 0.  , 0.  ],
       [0.41, 0.75, 0.55, 0.52, 0.05, 0.  , 0.06, 0.05, 0.05],
       [0.87, 0.95, 0.91, 0.82, 0.  , 0.  , 0.  , 0.  , 0.05],
       [0.92, 0.57, 0.4 , 0.31, 0.  , 0.05, 0.05, 0.  , 0.  ],
       [0.05, 0.09, 0.  , 0.  , 0.41, 0.37, 0.74, 0.07, 0.05],
       [0.  , 0.  , 0.09, 0.08, 0.46, 0.63, 0.5 , 0.05, 0.  ],
       [0.05, 0.1 , 0.  , 0.  , 0.44, 0.26, 0.17, 0.  , 0.  ],
       [0.  , 0.05, 0.  , 0.  , 0.  , 0.1 , 0.07, 0.99, 0.45],
       [0.09, 0.  , 0.07, 0.  , 0.  , 0.06, 0.  , 0.06, 0.33]])

### Lets get the Laplacian matrix L first. 

If A in R^{NxN}+ (positive reals) has K connected components, the rank of L is N-K. 

In [5]:
def laplacian(A):
  """
  My implementation; w_ij = -w_ij for i != j; w_ii = sum_{j != i} w_ij
  A bit slower than Scipy's for numpy arrays. 
  
  Works for both numpy array and torch tensor. 
  
  Note that this will be a differentiable function of A.
  Note that Laplacian at most rank n-1. 
  """
  eye = torch.eye if torch.is_tensor(A) else np.eye
  I = eye(len(A))
  return (I - 1) * A + I * ((1-I)*A).sum(0, keepdims=True)

assert np.all(laplacian(noisy_matrix) == csgraph.laplacian(noisy_matrix))
csgraph.laplacian(noisy_matrix)

array([[ 2.4 , -0.71, -0.63, -0.63, -0.01, -0.09, -0.01, -0.04, -0.01],
       [-0.41,  2.52, -0.55, -0.52, -0.05, -0.02, -0.06, -0.05, -0.05],
       [-0.87, -0.95,  1.76, -0.82, -0.  , -0.03, -0.02, -0.04, -0.05],
       [-0.92, -0.57, -0.4 ,  2.12, -0.01, -0.05, -0.05, -0.03, -0.  ],
       [-0.05, -0.09, -0.01, -0.  ,  0.99, -0.37, -0.74, -0.07, -0.05],
       [-0.01, -0.03, -0.09, -0.08, -0.46,  0.98, -0.5 , -0.05, -0.01],
       [-0.05, -0.1 , -0.  , -0.01, -0.44, -0.26,  1.48, -0.02, -0.  ],
       [-0.  , -0.05, -0.01, -0.02, -0.02, -0.1 , -0.07,  0.36, -0.45],
       [-0.09, -0.02, -0.07, -0.04, -0.  , -0.06, -0.03, -0.06,  0.62]])

In [6]:
At = torch.tensor(noisy_matrix)

In [7]:
L = laplacian(At)

### Now we're going to approximate $A$ by $\hat A$ and optimize laplacian of $\hat A$ to be low rank.

Doing this with Pytorch and gradient descent instead of how it is done in the paper http://openaccess.thecvf.com/content_cvpr_2014/papers/Feng_Robust_Subspace_Segmentation_2014_CVPR_paper.pdf (they use quadratic programming solver).

First make a function to find a low rank approx using PCA, which we will compute and use as a learning target. 
(another way might be to parameterize the low rank approximation as a matrix product) 

In [8]:
def low_rank_approx(A, rank):
  """
  Uses PCA to compute a low rank approximation to A.
  """
  assert rank <= len(A)
  U, S, V = torch.svd(A)
  return torch.mm(torch.mm(U[:,:rank], torch.diag(S[:rank])), V[:,:rank].t())

In [9]:
# Verify that this works
for i in range(1, len(L)):
  low_rank_L = low_rank_approx(L, i)
  print('Rank {} approximation error: {}'.format(torch.matrix_rank(low_rank_L), torch.sum((low_rank_L - L)**2)))

Rank 1 approximation error: 22.162927314392885
Rank 2 approximation error: 12.881264137512689
Rank 3 approximation error: 6.377761684976274
Rank 4 approximation error: 2.762220613383946
Rank 5 approximation error: 0.8777604546228892
Rank 6 approximation error: 0.2095245274252015
Rank 7 approximation error: 0.09926253369258037
Rank 8 approximation error: 6.533879114449026e-28


In [10]:
import torch.nn.functional as F

In [11]:
hat_At_sqrt = torch.sqrt(At.detach())
hat_At_sqrt.requires_grad_(True)
K = 3 # number of connected components
optimizer = torch.optim.Adam([hat_At_sqrt], lr=1e-3)

In [12]:
for i in range(1000):
  hat_At = hat_At_sqrt ** 2
  hat_L = laplacian(hat_At + hat_At.T)
  low_rank_target = low_rank_approx(hat_L, len(hat_L) - K)
  loss = F.l1_loss(hat_L, low_rank_target) + F.mse_loss(hat_At, At)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [13]:
(hat_At - At).detach().numpy().round(2)

array([[-0.  , -0.  ,  0.  ,  0.  , -0.01, -0.09, -0.01, -0.04, -0.01],
       [-0.  , -0.  , -0.  , -0.  , -0.05, -0.02, -0.06, -0.05, -0.05],
       [-0.  , -0.  ,  0.  , -0.  ,  0.  , -0.03, -0.02, -0.04, -0.05],
       [-0.  , -0.  ,  0.  , -0.  , -0.01, -0.05, -0.05, -0.03,  0.  ],
       [-0.05, -0.09, -0.01,  0.  ,  0.  , -0.  ,  0.  , -0.07, -0.05],
       [-0.01, -0.03, -0.09, -0.08,  0.  ,  0.  , -0.  , -0.05, -0.01],
       [-0.05, -0.1 ,  0.  , -0.01, -0.  , -0.  ,  0.  , -0.02,  0.  ],
       [ 0.  , -0.05, -0.01, -0.02, -0.02, -0.1 , -0.07,  0.  , -0.  ],
       [-0.09, -0.02, -0.07, -0.04,  0.  , -0.06, -0.03, -0.  ,  0.  ]])

## Behold the thresholded matrix:

In [14]:
hat_At.detach().numpy().round(2)

array([[0.52, 0.71, 0.63, 0.63, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.41, 0.75, 0.55, 0.52, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.87, 0.95, 0.91, 0.82, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.92, 0.57, 0.4 , 0.31, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.41, 0.37, 0.74, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.46, 0.63, 0.5 , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.44, 0.26, 0.17, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.45],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.06, 0.33]])

In [15]:
At.numpy().round(2)

array([[0.52, 0.71, 0.63, 0.63, 0.01, 0.09, 0.01, 0.04, 0.01],
       [0.41, 0.75, 0.55, 0.52, 0.05, 0.02, 0.06, 0.05, 0.05],
       [0.87, 0.95, 0.91, 0.82, 0.  , 0.03, 0.02, 0.04, 0.05],
       [0.92, 0.57, 0.4 , 0.31, 0.01, 0.05, 0.05, 0.03, 0.  ],
       [0.05, 0.09, 0.01, 0.  , 0.41, 0.37, 0.74, 0.07, 0.05],
       [0.01, 0.03, 0.09, 0.08, 0.46, 0.63, 0.5 , 0.05, 0.01],
       [0.05, 0.1 , 0.  , 0.01, 0.44, 0.26, 0.17, 0.02, 0.  ],
       [0.  , 0.05, 0.01, 0.02, 0.02, 0.1 , 0.07, 0.99, 0.45],
       [0.09, 0.02, 0.07, 0.04, 0.  , 0.06, 0.03, 0.06, 0.33]])